## 读取数据

In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
%matplotlib inline

In [3]:
def read_tsv(base_dir, app_name):
    file_names = glob.glob('{}/{}/*/*.tsv'.format(base_dir, app_name))
    dfs = []

    for file_name in file_names:
        df = pd.read_csv(file_name, sep='\t')
        dfs.append(df)

    frame = pd.concat(dfs, ignore_index=True)
    return frame

In [4]:
# DAU
dau = read_tsv('data/sample-data/section8/daily/dau', 'game-01')

In [5]:
dau.head()

,log_date,app_name,user_id
0,2013-05-01,game-01,608801
1,2013-05-01,game-01,712453
2,2013-05-01,game-01,776853
3,2013-05-01,game-01,823486
4,2013-05-01,game-01,113600


In [6]:
# DPU
dpu = read_tsv('data/sample-data/section8/daily/dpu', 'game-01')

In [7]:
dpu.head()

,log_date,app_name,user_id,payment
0,2013-05-01,game-01,804005,571
1,2013-05-01,game-01,793537,81
2,2013-05-01,game-01,317717,81
3,2013-05-01,game-01,317717,81
4,2013-05-01,game-01,426525,324


In [8]:
# 用户行为数据
user_action = read_tsv('data/sample-data/section8/daily/action', 'game-01')

In [9]:
user_action.head()

,log_date,app_name,user_id,A1,A2,A3,A4,A5,A6,A7,...,A45,A46,A47,A48,A49,A50,A51,A52,A53,A54
0,2013-10-31,game-01,654133,0,0,0,0,0,0,0,...,0,0,380,25655,0,0,0,0,0.0,46
1,2013-10-31,game-01,425530,0,0,0,0,10,1,233,...,19,20,180543,347,36,22,4,0,0.0,71
2,2013-10-31,game-01,709596,0,0,0,0,0,0,0,...,0,0,416,24817,0,0,0,0,0.0,2
3,2013-10-31,game-01,525047,0,2,0,0,9,0,0,...,22,22,35200,6412,21,0,0,0,0.0,109
4,2013-10-31,game-01,796908,0,0,0,0,0,0,0,...,29,29,388,25444,1,0,0,0,0.0,64


In [10]:
# 合并消费数据
dau2 = pd.merge(dau, dpu[['log_date', 'user_id', 'payment']], how='left', on=['log_date', 'user_id'])

In [11]:
dau2.head()

,log_date,app_name,user_id,payment
0,2013-05-01,game-01,608801,NaN
1,2013-05-01,game-01,712453,NaN
2,2013-05-01,game-01,776853,NaN
3,2013-05-01,game-01,823486,NaN
4,2013-05-01,game-01,113600,NaN


In [12]:
# 添加消费额标志
dau2['is_payment'] = np.where(dau2['payment'].isnull(), 0, 1)

In [13]:
dau2.head()

,log_date,app_name,user_id,payment,is_payment
0,2013-05-01,game-01,608801,NaN,0
1,2013-05-01,game-01,712453,NaN,0
2,2013-05-01,game-01,776853,NaN,0
3,2013-05-01,game-01,823486,NaN,0
4,2013-05-01,game-01,113600,NaN,0


In [14]:
# 讲无消费记录的消费额设为0
dau2['payment'] = np.where(dau2['payment'].isnull(), 0, dau2['payment'])

In [15]:
dau2.head()

,log_date,app_name,user_id,payment,is_payment
0,2013-05-01,game-01,608801,0.0,0
1,2013-05-01,game-01,712453,0.0,0
2,2013-05-01,game-01,776853,0.0,0
3,2013-05-01,game-01,823486,0.0,0
4,2013-05-01,game-01,113600,0.0,0


In [16]:
# 添加月份
dau2['log_month'] = dau2['log_date'].map(lambda x: x[:7])

In [17]:
dau2.head()

,log_date,app_name,user_id,payment,is_payment,log_month
0,2013-05-01,game-01,608801,0.0,0,2013-05
1,2013-05-01,game-01,712453,0.0,0,2013-05
2,2013-05-01,game-01,776853,0.0,0,2013-05
3,2013-05-01,game-01,823486,0.0,0,2013-05
4,2013-05-01,game-01,113600,0.0,0,2013-05


In [18]:
# 按月统计MAU
mau = dau2.groupby(['log_month', 'user_id']).agg({'payment': sum, 'user_id': 'count'})
mau = mau.rename(columns={'user_id': 'access_days'}).reset_index()

In [19]:
mau.head()

,log_month,user_id,payment,access_days
0,2013-05,65,0.0,1
1,2013-05,115,0.0,1
2,2013-05,194,0.0,1
3,2013-05,426,0.0,4
4,2013-05,539,0.0,1


## k-mean模型

In [21]:
from sklearn.cluster import KMeans

In [22]:
k = 3

In [23]:
km = KMeans(n_clusters=k, random_state=1, n_init=100)

In [24]:
km.fit(user_action[['A47']])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=100, n_jobs=None, precompute_distances='auto',
    random_state=1, tol=0.0001, verbose=0)

In [25]:
user_action2 = user_action.copy()

In [26]:
user_action2['cluster'] = km.labels_

In [28]:
user_action2['cluster'].value_counts()

0    2096
1     479
2      78
Name: cluster, dtype: int64

In [113]:
# list(km.labels_)

In [30]:
# 限定排名靠前的人
user_action_h = user_action2[user_action2['cluster'] >= 1]

In [31]:
user_action_h.head()

,log_date,app_name,user_id,A1,A2,A3,A4,A5,A6,A7,...,A46,A47,A48,A49,A50,A51,A52,A53,A54,cluster
1,2013-10-31,game-01,425530,0,0,0,0,10,1,233,...,20,180543,347,36,22,4,0,0.0,71,2
5,2013-10-31,game-01,776120,0,0,0,0,9,0,0,...,38,142214,684,37,15,0,0,0.0,312,2
7,2013-10-31,game-01,276197,0,0,0,0,7,0,58,...,15,54602,4226,15,0,8,0,0.0,95,1
8,2013-10-31,game-01,221572,0,0,0,0,1,0,0,...,24,39891,5792,4,0,0,0,0.0,21,1
9,2013-10-31,game-01,692433,0,0,0,0,6,0,0,...,28,50706,4549,16,8,0,0,0.0,154,1


## 进行主成分分析

In [32]:
user_action_f = user_action_h.set_index('user_id').loc[:, 'A2': 'A54']

In [33]:
user_action_f.head()

,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,...,A45,A46,A47,A48,A49,A50,A51,A52,A53,A54
user_id,,,,,,,,,,,,,,,,,,,,,
425530,0,0,0,10,1,233,58.25,288,230,19,...,19,20,180543,347,36,22,4,0,0.0,71
776120,0,0,0,9,0,0,0.00,325,195,38,...,19,38,142214,684,37,15,0,0,0.0,312
276197,0,0,0,7,0,58,7.25,150,100,15,...,15,15,54602,4226,15,0,8,0,0.0,95
221572,0,0,0,1,0,0,0.00,40,14,0,...,24,24,39891,5792,4,0,0,0,0.0,21
692433,0,0,0,6,0,0,0.00,102,95,0,...,15,28,50706,4549,16,8,0,0,0.0,154


In [34]:
from sklearn.feature_selection import VarianceThreshold

In [35]:
# 删除信息量小的
vt = VarianceThreshold()
vt.fit(user_action_f)

VarianceThreshold(threshold=0.0)

In [36]:
vt.get_support().sum()

45

In [37]:
user_action_f_filterd = user_action_f.iloc[:, vt.get_support()].copy()

In [38]:
def freq_cut(df):
    cols = []
    
    for i in df.columns:
        value_count = df[i].value_counts().to_list()
        f1 = ((value_count[0] / value_count[1]) <= (95/5))
        f2 = ((df[i].nunique() / len(df[i])) >= 0.1)
        if f1 or f2:
            cols.append(i)
    return df[cols]

In [39]:
user_action_f_filterd = freq_cut(user_action_f_filterd)

In [40]:
user_action_f_filterd.head()

,A2,A5,A6,A7,A8,A9,A10,A11,A12,A13,...,A45,A46,A47,A48,A49,A50,A51,A52,A53,A54
user_id,,,,,,,,,,,,,,,,,,,,,
425530,0,10,1,233,58.25,288,230,19,2,19,...,19,20,180543,347,36,22,4,0,0.0,71
776120,0,9,0,0,0.00,325,195,38,8,19,...,19,38,142214,684,37,15,0,0,0.0,312
276197,0,7,0,58,7.25,150,100,15,3,11,...,15,15,54602,4226,15,0,8,0,0.0,95
221572,0,1,0,0,0.00,40,14,0,0,3,...,24,24,39891,5792,4,0,0,0,0.0,21
692433,0,6,0,0,0.00,102,95,0,0,2,...,15,28,50706,4549,16,8,0,0,0.0,154


In [41]:
user_action_cor = user_action_f_filterd.corr() #.apply(lambda x: abs(x))

In [42]:
# user_action_cor

In [43]:
import itertools

In [44]:
feature_group = list(itertools.combinations(user_action_f_filterd.columns, 2))

In [47]:
# 删除相关性高的变量
def filter_corr(cutoff=0.7):
    cols = []
    for i,j in feature_group:
        if user_action_cor.loc[i, j] > cutoff:
#             cols.append((i, j))
#             print(i, j)
            i_avg = user_action_cor[i][user_action_cor[i] != 1].mean()
            j_avg = user_action_cor[j][user_action_cor[j] != 1].mean()
            if i_avg >= j_avg:
                cols.append(i)
            else:
                cols.append(j)
    return set(cols)

In [48]:
filter_corr()

{'A10', 'A15', 'A18', 'A26', 'A46', 'A49', 'A53', 'A6', 'A7', 'A9'}

In [49]:
ls = ['A49', 'A9', 'A15', 'A16', 'A10', 'A18', 'A48', 'A46', 'A26', 'A7', 'A23', 'A52']

In [50]:
features = ['A2', 'A5', 'A6', 'A8', 'A11', 'A12', 'A13', 'A14', 'A17', 'A20', 'A25', 'A43', 'A44', 'A45', 'A47', 'A50', 'A51', 'A53', 'A54']

In [51]:
user_action_f_filterd = user_action_f_filterd[features]

In [52]:
user_action_f_filterd.head()

,A2,A5,A6,A8,A11,A12,A13,A14,A17,A20,A25,A43,A44,A45,A47,A50,A51,A53,A54
user_id,,,,,,,,,,,,,,,,,,,
425530,0,10,1,58.25,19,2,19,13,6,0,0.92857,23,0.92174,19,180543,22,4,0.0,71
776120,0,9,0,0.00,38,8,19,10,11,0,0.92424,20,0.90256,19,142214,15,0,0.0,312
276197,0,7,0,7.25,15,3,11,5,3,0,0.83333,10,0.92000,15,54602,0,8,0.0,95
221572,0,1,0,0.00,0,0,3,0,1,0,0.90909,2,0.85714,24,39891,0,0,0.0,21
692433,0,6,0,0.00,0,0,2,2,5,0,1.00000,11,0.73684,15,50706,8,0,0.0,154


In [53]:
from sklearn.decomposition import PCA

In [54]:
pca = PCA(whiten=True) # 
pca.fit(user_action_f_filterd)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=True)

In [55]:
# 进行主成分分析
user_action_pca_base = pd.DataFrame(pca.transform(user_action_f_filterd), columns=user_action_f_filterd.columns)

In [56]:
user_action_pca_base.head()

,A2,A5,A6,A8,A11,A12,A13,A14,A17,A20,A25,A43,A44,A45,A47,A50,A51,A53,A54
0,1.774270,1.321898,-1.375938,1.744821,0.072420,0.684892,0.214132,2.936224,0.658769,-0.181322,-0.679511,1.927964,-0.735461,-0.676144,-0.480835,0.760232,-0.766946,0.089761,0.232928
1,1.057669,-2.154146,-0.547395,-0.848810,-0.349375,2.413903,-0.114411,1.395555,0.154607,-0.560008,0.458182,-1.582546,0.866935,-0.608628,-1.322319,0.191498,0.094717,-0.163687,0.323289
2,-0.580337,0.153276,0.106240,-0.088895,0.082244,0.463854,-0.419742,-0.498646,0.440438,2.392561,1.517827,0.143198,-0.830166,-0.626232,-0.105983,0.267950,-0.066550,-0.424313,0.845666
3,-0.855376,1.038575,0.214495,-0.649453,0.330824,-2.031787,-0.268538,-0.120757,0.141404,-0.102384,-0.421732,-0.674522,-0.517404,0.016909,-0.213880,0.231475,-0.170792,-0.837950,-0.117414
4,-0.653176,-0.633376,-0.371262,-0.818966,1.382758,0.247161,0.442689,1.229008,0.245788,-0.555424,0.698416,-0.005929,0.535366,-0.356558,-0.932452,0.331640,-0.390932,-0.691750,-1.390338


In [57]:
user_action_pca_base.shape

(557, 19)

## 进行聚类

In [65]:
user_action_km = KMeans(n_clusters=5, random_state=0, n_init=100)

In [66]:
user_action_km.fit(user_action_pca_base) # [['A2', 'A11', 'A13', 'A43', 'A44', 'A51']]

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=100, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [67]:
np.bincount(user_action_km.labels_)

array([ 25,  38, 189,  86, 219], dtype=int64)

In [68]:
## 计算每个类的平均值
user_action_f_filterd['cluster'] = user_action_km.labels_

In [69]:
# user_action_f_filterd

In [70]:
user_action_f_center = user_action_f_filterd.groupby(['cluster']).mean()

In [71]:
user_action_f_center

,A2,A5,A6,A8,A11,A12,A13,A14,A17,A20,A25,A43,A44,A45,A47,A50,A51,A53,A54
cluster,,,,,,,,,,,,,,,,,,,
0,0.000000,1.080000,0.200000,1.133600,0.040000,0.240000,6.440000,0.720000,0.640000,0.000000,0.000000,1.360000,0.243674,5.360000,63691.440000,0.400000,0.360000,0.000000,77.880000
1,1.157895,5.868421,0.394737,18.958158,12.868421,2.289474,9.421053,6.947368,6.026316,1.052632,0.913577,13.552632,0.878269,20.842105,59471.421053,4.815789,1.947368,0.095645,84.631579
2,0.000000,6.021164,0.412698,10.853386,22.465608,4.201058,21.021164,8.396825,5.862434,0.216931,0.902391,18.465608,0.873082,23.571429,112024.312169,6.597884,1.566138,0.000000,153.439153
3,0.034884,5.430233,0.720930,14.326163,14.406977,2.465116,14.116279,6.127907,5.023256,0.337209,0.906638,17.488372,0.864259,20.046512,86482.034884,4.406977,2.232558,0.942862,95.488372
4,0.000000,4.817352,0.337900,8.707808,15.520548,1.593607,15.438356,4.922374,3.356164,0.260274,0.924568,9.077626,0.844629,22.410959,69591.867580,3.835616,1.415525,0.000000,115.913242
